In [3]:
import numpy as np
import pandas as pd

# Normalized Discounted Cumulative Gain

In [24]:
# --- CLASS QUERY RELEVANCE ---

# Query: className:Sorter;methods:sort
sorter_relevance = [1, 2, 3, 1, 0, 0, 0, 0, 0, 0]

# Query: className:Piece;methods:move,getPosition,getPos,getX,getY
piece_relevance = [2, 2, 2, 1, 3, 2, 1, 1, 1, 3]

# --- METHOD QUERY RELEVANCE ---

# Query: methodName:hash;returnType:String;modifiers:static;parameters:input-String
hash_relevance = [3, 3, 2, 3, 0, 2, 1, 1, 1, 1]

# Query: methodName:reverse;returnType:String;parameters:input-String,str-String
reverse_relevance = [3, 3, 0, 0, 1, 1, 1, 0, 1, 1]


In [14]:
# Calculate Normalized Discounted Cumulative Gain (NDCG)

def ndcg_score(relevance):
    ndgc = pd.DataFrame(columns = ['rel_score', 'CG', 'DCG', 'ideal_system', 'ICG', 'IDCG', 'NDCG'])
    ndgc['rel_score'] = relevance
    

    ideal = relevance.copy()
    ideal.sort(reverse = True)

    ndgc['ideal_system'] = ideal
    
    ## Calculate the cumulative gain
    ndgc['CG'] = ndgc['rel_score'].cumsum()
    ndgc['ICG'] = ndgc['ideal_system'].cumsum()

    ## Calculate the discounted cumulative gain
    for i in range(len(ndgc)):
        if i == 0:
            ndgc.loc[i, 'DCG'] = float(ndgc.loc[i, 'CG'])
            ndgc.loc[i, 'IDCG'] = float(ndgc.loc[i, 'ICG'])

        else:
            ndgc.loc[i, 'DCG'] = ndgc.loc[i-1, 'DCG'] + ndgc.loc[i, 'rel_score'] / np.log2(i+2)
            ndgc.loc[i, 'IDCG'] = ndgc.loc[i-1, 'IDCG'] + ndgc.loc[i, 'ideal_system'] / np.log2(i+2)


    ndgc['NDCG'] = ndgc['DCG'] / ndgc['IDCG']
    
    return ndgc


In [22]:
ndgc = ndcg_score(sorter_relevance)
ndgc.head(13)

,rel_score,CG,DCG,ideal_system,ICG,IDCG,NDCG
0,1,1,1.0,3,3,3.0,0.333333
1,2,3,2.26186,2,5,4.26186,0.530721
2,3,6,3.76186,1,6,4.76186,0.789998
3,1,7,4.192536,1,7,5.192536,0.807416
4,0,7,4.192536,0,7,5.192536,0.807416
5,0,7,4.192536,0,7,5.192536,0.807416
6,0,7,4.192536,0,7,5.192536,0.807416
7,0,7,4.192536,0,7,5.192536,0.807416
8,0,7,4.192536,0,7,5.192536,0.807416
9,0,7,4.192536,0,7,5.192536,0.807416


In [17]:
ndgc = ndcg_score(piece_relevance)
ndgc.head(13)

,rel_score,CG,DCG,ideal_system,ICG,IDCG,NDCG
0,2,2,2.0,3,3,3.0,0.666667
1,2,4,3.26186,3,6,4.892789,0.666667
2,2,6,4.26186,2,8,5.892789,0.723233
3,1,7,4.692536,2,10,6.754142,0.694764
4,3,10,5.853094,2,12,7.527848,0.777526
5,2,12,6.565509,2,14,8.240262,0.79676
6,1,13,6.898842,1,15,8.573596,0.804661
7,1,14,7.214307,1,16,8.889061,0.811594
8,1,15,7.515337,1,17,9.190091,0.817765
9,3,18,8.382532,1,18,9.479155,0.884312


In [20]:
ndcg_score(hash_relevance).head(13)

,rel_score,CG,DCG,ideal_system,ICG,IDCG,NDCG
0,3,3,3.0,3,3,3.0,1.0
1,3,6,4.892789,3,6,4.892789,1.0
2,2,8,5.892789,3,9,6.392789,0.921787
3,3,11,7.184819,2,11,7.254142,0.990444
4,0,11,7.184819,2,13,8.027848,0.894987
5,2,13,7.897233,1,14,8.384055,0.941935
6,1,14,8.230567,1,15,8.717389,0.944155
7,1,15,8.546032,1,16,9.032853,0.946105
8,1,16,8.847062,1,17,9.333883,0.947844
9,1,17,9.136126,0,17,9.333883,0.978813


In [25]:
ndcg_score(reverse_relevance).head(13)

,rel_score,CG,DCG,ideal_system,ICG,IDCG,NDCG
0,3,3,3.0,3,3,3.0,1.0
1,3,6,4.892789,3,6,4.892789,1.0
2,0,6,4.892789,1,7,5.392789,0.907284
3,0,6,4.892789,1,8,5.823466,0.840185
4,1,7,5.279642,1,9,6.210319,0.85014
5,1,8,5.635849,1,10,6.566526,0.85827
6,1,9,5.969183,1,11,6.899859,0.865117
7,0,9,5.969183,0,11,6.899859,0.865117
8,1,10,6.270213,0,11,6.899859,0.908745
9,1,11,6.559277,0,11,6.899859,0.950639
